In [5]:

import logging

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p"
)

logger = logging.getLogger(__name__)

class PegandoUrl:
    def __init__(self, url: str):
        self.url = url
        
    def options(self):
        options = Options()
        #options.add_argument("--headless")  # Executar em modo headless
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        # User agent realista
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

        # Outras configurações
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920,1080")
        return options
        
    def webdriver(self):
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options()
        )
        return driver

    def get_url(self):
        driver = self.webdriver()
        driver.get(self.url) 
        return driver

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

class ZapImoveisScraperBase:
    """Classe base para fazer scraping do Zap Imóveis com Selenium"""
    
    def __init__(self, headless: bool = False, wait_time: int = 5):
        """
        Inicializa o scraper
        
        Args:
            headless: Se True, executa em modo headless
            wait_time: Tempo de espera em segundos após carregar a página
        """
        self.wait_time = wait_time
        self.driver = None
        self.options = self._configure_options(headless)
    
    def _configure_options(self, headless: bool) -> Options:
        """Configura as opções do Chrome"""
        options = Options()
        
        # Remover indicadores de automação
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        
        # User agent realista
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        # Configurações gerais
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920,1080")
        
        if headless:
            options.add_argument("--headless")
        
        return options
    
    def _initialize_driver(self) -> webdriver.Chrome:
        """Inicializa e configura o driver do Chrome"""
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.options
        )
        
        # Remover propriedade webdriver
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        return driver
    
    def get_page_source(self, url: str) -> str:
        """
        Obtém o HTML de uma página
        
        Args:
            url: URL da página a scrappear
            
        Returns:
            String contendo o HTML da página
        """
        self.driver = self._initialize_driver()
        
        try:
            self.driver.get(url)
            time.sleep(self.wait_time)
            html = self.driver.page_source
            return html
        finally:
            self.driver.quit()
    
    def close(self):
        """Fecha o driver se estiver aberto"""
        if self.driver:
            self.driver.quit()


In [7]:
inst = ZapImoveisScraperBase(headless=True)

html = inst.get_page_source("https://www.zapimoveis.com.br/venda/?pagina=2&transacao=Venda")

02/11/2026 01:34:26 PM ====== WebDriver manager ======
02/11/2026 01:34:28 PM Get LATEST chromedriver version for google-chrome
02/11/2026 01:34:28 PM Get LATEST chromedriver version for google-chrome
02/11/2026 01:34:29 PM Driver [C:\Users\jefer\.wdm\drivers\chromedriver\win64\144.0.7559.133\chromedriver-win32/chromedriver.exe] found in cache


In [ ]:
html

'<html lang="pt-BR"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1"><link rel="preload" as="image" href="https://resizedimgs.zapimoveis.com.br/img/vr-listing/98d0ee93e27f8f049faa6365104c9de3/apartamento-com-3-quartos-a-venda-100m-no-sion-belo-horizonte.webp?action=fit-in&amp;dimension=614x297" fetchpriority="high"><link rel="stylesheet" href="https://cdn-rp-zap-ssr-prod.zapimoveis.com.br/_next/static/chunks/dd67ce31ada98833.css" data-precedence="next"><link rel="stylesheet" href="https://cdn-rp-zap-ssr-prod.zapimoveis.com.br/_next/static/chunks/210a2413f6b7d313.css" data-precedence="next"><link rel="stylesheet" href="https://cdn-rp-zap-ssr-prod.zapimoveis.com.br/_next/static/chunks/108915c077ab7281.css" data-precedence="next"><link rel="stylesheet" href="https://cdn-rp-zap-ssr-prod.zapimoveis.com.br/_next/static/chunks/75a3516923e1303d.css" data-precedence="next"><link rel="stylesheet" href="https://cdn-rp-zap-ssr-prod.zapimoveis.com.br/_

: 

In [4]:
soup = BeautifulSoup(html, 'html.parser')

In [5]:
from bs4 import BeautifulSoup
from typing import List

class ZapImoveisListingLinks:
    
    def __init__(self, soup: str, card_selector: dict):
        
        self.soup = soup
        self.CARD_SELECTOR = card_selector
    def extract_links(self) -> List[str]:
        """
        Extrai os links de imóveis do HTML da página de listagem
        
        Args:
            html: String contendo o HTML da página
            
        Returns:
            Lista com os URLs dos imóveis encontrados
        """
        cards = self.soup.find_all('li', self.CARD_SELECTOR)
        links = []
        
        for idx, card in enumerate(cards):
            link_tag = card.find(href=True)
            
            if link_tag and 'href' in link_tag.attrs:
                link = link_tag['href']
                
                # Completar URL relativa
                if not link.startswith('http'):
                    link = self.BASE_URL + link
                
                links.append(link)
            else:
                print(f"⚠️ Link não encontrado no card {idx + 1}")
        return links

        
    

In [7]:
inst = ZapImoveisScraperBase(headless=True)

html = inst.get_page_source("https://www.zapimoveis.com.br/venda/?pagina=2&transacao=Venda")

extrator = ZapImoveisListingLinks(
    soup = soup,
    card_selector={'data-cy': 'rp-property-cd'}
)

# Extrair links de uma única página
links = extrator.extract_links()

print(links)

02/11/2026 09:19:49 AM ====== WebDriver manager ======
02/11/2026 09:19:51 AM Get LATEST chromedriver version for google-chrome
02/11/2026 09:19:52 AM Get LATEST chromedriver version for google-chrome
02/11/2026 09:19:52 AM Driver [C:\Users\jefer\.wdm\drivers\chromedriver\win64\144.0.7559.133\chromedriver-win32/chromedriver.exe] found in cache


⚠️ Link não encontrado no card 16
⚠️ Link não encontrado no card 26
['https://www.zapimoveis.com.br/imovel/venda-sobrados-3-quartos-com-cozinha-parque-penha-zona-leste-sao-paulo-sp-115m2-id-2815677438/?source=ranking%2Crp', 'https://www.zapimoveis.com.br/imovel/venda-cobertura-3-quartos-paraiso-santo-andre-sp-134m2-id-2866886768/?source=ranking%2Crp', 'https://www.zapimoveis.com.br/imovel/venda-cobertura-4-quartos-com-churrasqueira-tijuca-zona-norte-rio-de-janeiro-rj-156m2-id-2801028720/?source=ranking%2Crp', 'https://www.zapimoveis.com.br/imovel/venda-terreno-lote-condominio-vila-maria-lages-896m2-id-2868946036/?source=ranking%2Crp', 'https://www.zapimoveis.com.br/imovel/venda-apartamento-2-quartos-com-churrasqueira-camobi-santa-maria-52m2-id-2868858475/?source=ranking%2Crp', 'https://www.zapimoveis.com.br/imovel/venda-apartamento-2-quartos-com-piscina-chacaras-residenciais-santa-maria-votorantim-55m2-id-2856592758/?source=ranking%2Crp', 'https://www.zapimoveis.com.br/imovel/venda-ter

In [8]:
links

['https://www.zapimoveis.com.br/imovel/venda-sobrados-3-quartos-com-cozinha-parque-penha-zona-leste-sao-paulo-sp-115m2-id-2815677438/?source=ranking%2Crp',
 'https://www.zapimoveis.com.br/imovel/venda-cobertura-3-quartos-paraiso-santo-andre-sp-134m2-id-2866886768/?source=ranking%2Crp',
 'https://www.zapimoveis.com.br/imovel/venda-cobertura-4-quartos-com-churrasqueira-tijuca-zona-norte-rio-de-janeiro-rj-156m2-id-2801028720/?source=ranking%2Crp',
 'https://www.zapimoveis.com.br/imovel/venda-terreno-lote-condominio-vila-maria-lages-896m2-id-2868946036/?source=ranking%2Crp',
 'https://www.zapimoveis.com.br/imovel/venda-apartamento-2-quartos-com-churrasqueira-camobi-santa-maria-52m2-id-2868858475/?source=ranking%2Crp',
 'https://www.zapimoveis.com.br/imovel/venda-apartamento-2-quartos-com-piscina-chacaras-residenciais-santa-maria-votorantim-55m2-id-2856592758/?source=ranking%2Crp',
 'https://www.zapimoveis.com.br/imovel/venda-terreno-lote-condominio-jardim-rio-negro-itaquaquecetuba-940m2-id

In [9]:
inst_dados = ZapImoveisScraperBase(headless=True)

html_dados = inst.get_page_source(links[0])

soup_dados = BeautifulSoup(html_dados, 'html.parser')

02/11/2026 09:20:11 AM ====== WebDriver manager ======
02/11/2026 09:20:13 AM Get LATEST chromedriver version for google-chrome
02/11/2026 09:20:14 AM Get LATEST chromedriver version for google-chrome
02/11/2026 09:20:14 AM Driver [C:\Users\jefer\.wdm\drivers\chromedriver\win64\144.0.7559.133\chromedriver-win32/chromedriver.exe] found in cache


In [10]:
soup_dados = BeautifulSoup(html_dados, 'html.parser')

In [11]:
class ZapImoveisPhotoLinks:
    def __init__(self, soup: str):
        
        self.soup = soup
    
    def extract_photos(self) -> List[str]:
        items = self.soup.find_all('li', class_='carousel-photos--item')

        fotos = []
        for item in items:
            source = item.find('source')
            if source and 'srcset' in source.attrs:
                fotos.append(source['srcset'].split()[0])
        return fotos

fotos = ZapImoveisPhotoLinks(soup_dados).extract_photos()

In [12]:
class ZapImoveisAmenities:
    def __init__(self, soup: str):
        self.soup = soup
    def extract_amenities(self) -> dict:
        amenities_list = self.soup.find('ul', class_='amenities-scrollbar')
        return amenities_list
    

amenities = ZapImoveisAmenities(soup_dados).extract_amenities()


class ZapImoveisMetragem:
    def __init__(self, amenities: str):
        self.amenities = amenities
    
    def extract_metragem(self) -> str:
        metragem = self.amenities.find('p', string='Metragem').find_next('p', class_='font-bold').get_text(strip=True)   
        return metragem
    
    def limpar_metragem(self) -> float:
        metragem = self.extract_metragem()
        return float(metragem.replace('m²', '').strip())

class ZapImoveisQuartos:
    def __init__(self, amenities: str):
        self.amenities = amenities
    
    def extract_quartos(self) -> int:
        quartos = self.amenities.find('p', string='Quartos').find_next('p', class_='font-bold').get_text(strip=True)   
        return int(quartos)

class ZapImoveisBanheiros:
    def __init__(self, amenities: str):
        self.amenities = amenities
    
    def extract_banheiros(self) -> int:
        banheiros = self.amenities.find('p', string='Banheiros').find_next('p', class_='font-bold').get_text(strip=True)   
        return int(banheiros)

class ZapImoveisVagasGaragem:
    def __init__(self, amenities: str):
        self.amenities = amenities
    
    def extract_vagas(self) -> int:
        vagas = self.amenities.find('p', string='Vagas').find_next('p', class_='font-bold').get_text(strip=True)   
        return int(vagas)
    
      

In [ ]:
"""import unicodedata
class ZapImoveisEndereco:
    def __init__(self, soup: str):
        self.soup = soup    
    
    def extract_endereco(self) -> str:
        endereco = self.soup.find('p', {'data-testid': 'location-address'}).get_text(strip=True)
        return endereco
    
    def explit_endereco(self) -> str:
        endereco = self.extract_endereco()
        endereco_split = endereco.split(',')
        return endereco_split
    
    def explit_rua_bairro(self):
        endereco_split = self.explit_endereco()
        rua_bairro = endereco_split[0]
        rua_bairro_split = rua_bairro.split('-')
        rua = rua_bairro_split[0].strip()
        bairro = rua_bairro_split[1].strip()
        return rua, bairro
    
    def limpar_rua(self) -> str:
        rua, _ = self.explit_rua_bairro()
        if 'rua' in rua:
            rua = rua.split('rua')[1].strip()
        elif 'avenida' in rua:
            rua = rua.split('avenida')[1].strip()
        elif 'estrada' in rua:
            rua = rua.split('estrada')[1].strip()
        return rua.lower()
    
    def limpar_bairro(self) -> str:
        _ , bairro = self.explit_rua_bairro()
        if ' - ' in bairro:
            bairro = bairro.split(' - ')[1].strip()
        return bairro.lower()
    
    def extract_cidade_estado(self) -> str:
        endereco_split = self.explit_endereco()
        cidade_estado = endereco_split[2].strip()
        cidade = cidade_estado.split('-')[0].strip()
        estado = cidade_estado.split('-')[1].strip()
        return cidade.lower(), estado
    
    def endereco_completo(self) -> str:
        rua = self.limpar_rua()
        bairro = self.limpar_bairro()
        cidade, estado = self.extract_cidade_estado()
        return rua, bairro, cidade, estado"""

In [13]:
import unicodedata
class ZapImoveisEndereco:
    def __init__(self, soup: str):
        self.soup = soup    
    
    def extract_endereco(self) -> str:
        endereco = self.soup.find('p', {'data-testid': 'location-address'}).get_text(strip=True)
        return endereco
    
    def explit_endereco(self) -> list:
        endereco = self.extract_endereco()
        endereco_split = endereco.split(',')
        return endereco_split
    
    def explit_rua_bairro(self):
        
        endereco_split = self.explit_endereco()
        
        # Formato 1: ['Rua X', ' 351 - Boa Vista', ' Marília - SP'] - 3 partes
        # Formato 2: ['Avenida X - Butantã', ' São Paulo - SP'] - 2 partes
        
        if len(endereco_split) == 3:
            # Tem número separado
            rua = endereco_split[0].strip()
            bairro_parte = endereco_split[1].strip()  # "351 - Boa Vista"
            
            # Pega só o bairro (depois do hífen)
            if ' - ' in bairro_parte:
                bairro = bairro_parte.split(' - ')[-1].strip()
            else:
                bairro = bairro_parte
                
        elif len(endereco_split) == 2:
            # Rua e bairro juntos
            rua_bairro = endereco_split[0].strip()  # "Avenida X - Butantã"
            
            if ' - ' in rua_bairro:
                partes = rua_bairro.split(' - ')
                rua = partes[0].strip()
                bairro = partes[-1].strip()  # Pega o último (bairro)
            else:
                rua = rua_bairro
                bairro = ""
        else:
            # Caso inesperado
            rua = endereco_split[0].strip() if endereco_split else ""
            bairro = ""
        
        return rua, bairro
    
    def limpar_rua(self) -> str:
        rua, _ = self.explit_rua_bairro()
        rua_lower = rua.lower()
        
        if 'rua ' in rua_lower:
            rua = rua_lower.split('rua ')[1].strip()
        elif 'avenida ' in rua_lower:
            rua = rua_lower.split('avenida ')[1].strip()
        elif 'estrada ' in rua_lower:
            rua = rua_lower.split('estrada ')[1].strip()
        else:
            rua = rua_lower
            
        return unicodedata.normalize('NFD', rua).encode('ascii', 'ignore').decode('utf-8')
    
    def limpar_bairro(self) -> str:
        _, bairro = self.explit_rua_bairro()
        
        bairro_limpo = unicodedata.normalize('NFD', bairro).encode('ascii', 'ignore').decode('utf-8')
        
        return bairro_limpo.lower()
    
    def extract_cidade_estado(self) -> tuple:
        endereco_split = self.explit_endereco()
        cidade_estado = endereco_split[-1].strip()  # Último elemento
        
        if ' - ' in cidade_estado:
            partes = cidade_estado.split(' - ')
            cidade = partes[0].strip()
            estado = partes[1].strip() if len(partes) > 1 else ""
        else:
            cidade = cidade_estado
            estado = ""
           
        
        return cidade.lower(), estado.upper()
    
    def endereco_completo(self) -> tuple:
        try:
            rua = self.limpar_rua()
            rua = unicodedata.normalize('NFD', rua).encode('ascii', 'ignore').decode('utf-8')
        except:
            rua = "s/n"
        try:
            bairro = self.limpar_bairro()
        except:
            bairro = "s/n"
        try:
            cidade, estado = self.extract_cidade_estado()
            cidade = unicodedata.normalize('NFD', cidade).encode('ascii', 'ignore').decode('utf-8')
            estado = unicodedata.normalize('NFD', estado).encode('ascii', 'ignore').decode('utf-8')
        except:
            bairro = "s/n"
        return rua, bairro, cidade, estado

In [16]:
metragem = ZapImoveisMetragem(amenities).limpar_metragem()
print("Metragem:", metragem)

quartos = ZapImoveisQuartos(amenities).extract_quartos()
print("Quartos:", quartos)

banheiros = ZapImoveisBanheiros(amenities).extract_banheiros()  
print("Banheiros:", banheiros)

garagen = ZapImoveisVagasGaragem(amenities).extract_vagas()
print("Vagas de garagem:", garagen)

endereco = ZapImoveisEndereco(soup_dados)
rua, bairro, cidade, estado = endereco.endereco_completo()

print("Rua:", rua)
print("Bairro:", bairro)
print("Cidade:", cidade)
print("Estado:", estado)

Metragem: 115.0
Quartos: 3
Banheiros: 3
Vagas de garagem: 2
Rua: nova cruz
Bairro: parque penha
Cidade: sao paulo
Estado: SP


In [17]:
class ZapImoveisTitulo:
    def __init__(self, soup: str):
        self.soup = soup
    
    def extract_titulo(self) -> str:
        titulo = self.soup.find('h2', class_='text-neutral-130').get_text(strip=True)
        return titulo

In [18]:
titulo = ZapImoveisTitulo(soup_dados).extract_titulo()

In [19]:
from datetime import datetime
import locale
class ZapImoveisDataPublicacao:
    def __init__(self, soup: str):
        self.soup = soup
    
    def extract_data_publicacao(self) -> str:
        
        data_criacao_info = self.soup.find('span', {'data-testid': 'listing-created-date'}).get_text(" ", strip=True)
        return data_criacao_info  
    
    def limpar_data_publicacao(self) -> str:    
        data_publicacao = self.extract_data_publicacao()
        data_publicacao_limpa = data_publicacao.split(',')[0].strip().split('Anúncio criado em')[1].strip()
        return data_publicacao_limpa
    
    def converter_para_data(self) -> str:
        try:
            locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')
        
        except:
            locale.setlocale(locale.LC_TIME, 'portuguese')
        
        data_publicacao = self.limpar_data_publicacao()
        data_objeto = datetime.strptime(data_publicacao, '%d de %B de %Y')
        data_formatada = data_objeto.strftime('%d/%m/%Y')
        return data_formatada
        
    

In [89]:
data = ZapImoveisDataPublicacao(soup_dados)
data_publicacao = data.converter_para_data()
print("Data de publicação:", data_publicacao)

Data de publicação: 26/11/2025


In [21]:
class ZapImoveisValorVenda:
    def __init__(self, soup: str):
        self.soup = soup
    
    def extract_valor(self) -> str:
        venda_elem = self.soup.find('p', string='Venda')
        venda_valor = venda_elem.find_parent('div').find_next_sibling('p').get_text(strip=True) if venda_elem else "N/A"
        return venda_valor
    
    def limpar_valor_venda(self) -> float:
        valor_venda = self.extract_valor()
        valor_venda_limpo = valor_venda.replace('R$', '').replace('.', '').strip()
        return float(valor_venda_limpo)

In [22]:
venda = ZapImoveisValorVenda(soup_dados).limpar_valor_venda()
print("Valor de venda:", venda)

Valor de venda: 520000.0


In [23]:
class ZapImoveisValorCondominio:
    def __init__(self, soup: str):
        self.soup = soup
    
    def extract_valor_condominio(self) -> str:
        condominio_elem = self.soup.find('p', {'data-testid': 'condoFee'})
        condominio_valor = condominio_elem.get_text(strip=True) if condominio_elem else "N/A"
        return condominio_valor
    
    def limpar_valor_condominio(self) -> float:
        valor_condominio = self.extract_valor_condominio()
        if valor_condominio == "N/A" or valor_condominio.lower().strip() == "isento":
            return 0
        elif valor_condominio.lower().strip() == "não informado":
            return "nao informado"
        valor_condominio_limpo = valor_condominio.replace('R$', '').replace('.', '').strip()
        
        if "/mês" in valor_condominio_limpo:
            valor_condominio_limpo = valor_condominio_limpo.replace('/mês', '').strip()
        return valor_condominio_limpo
    
condominio = ZapImoveisValorCondominio(soup_dados)
valor_condominio = condominio.limpar_valor_condominio()


In [24]:
class ZapImoveisIPTU:
    def __init__(self, soup: str):
        self.soup = soup
    
    def extract_valor_iptu(self) -> str:
        iptu_elem = self.soup.find('p', {'data-testid': 'iptu'})
        iptu_valor = iptu_elem.get_text(strip=True) if iptu_elem else "N/A"
        return iptu_valor

    def limpar_valor_iptu(self) -> float:
        valor_iptu = self.extract_valor_iptu()
        if valor_iptu.lower().strip() == "não informado":
            return "nao informado"
        elif valor_iptu.lower().strip() == "isento":
            return "isento"
        valor_iptu_limpo = valor_iptu.replace('R$', '').replace('.', '').strip()
        
        if "/mês" in valor_iptu_limpo:
            valor_iptu_limpo = valor_iptu_limpo.replace('/mês', '').strip()
        return valor_iptu_limpo

iptu = ZapImoveisIPTU(soup_dados).limpar_valor_iptu()
print("Valor do IPTU:", iptu)

Valor do IPTU: isento


In [25]:
class ZapImoveisCaracteristicas:
    def __init__(self, soup: str):
        self.soup = soup
    
    def extract_amenities(self) -> str:
        amenities_elem = self.soup.find('ul', {'data-testid': 'amenities-list'})
        items = amenities_elem.find_all('li')
        return items
    def extract_caracteristicas(self):
        items = self.extract_amenities()
        caracteristicas = {}

        for li in items:
            # O nome da característica (ex: floorLevel, PETS_ALLOWED)
            chave = li.get('itemprop')
            # O texto visível (ex: 2 andar, Aceita animais)
            valor = li.find('span', class_='amenities-item-text').get_text(strip=True)
            
            caracteristicas[chave] = valor
        return caracteristicas

In [26]:
amenities = ZapImoveisCaracteristicas(soup_dados).extract_caracteristicas()
amenities

{'floorSize': '115 m²',
 'numberOfRooms': '3 quartos',
 'numberOfBathroomsTotal': '3 banheiros',
 'numberOfParkingSpaces': '2 vagas',
 'numberOfSuites': '1 suíte',
 'PETS_ALLOWED': 'Aceita animais',
 'BACKYARD': 'Quintal',
 'KITCHEN': 'Cozinha',
 'COPA': 'Copa'}

In [27]:
class ZapImoveisDescricao:
    def __init__(self, soup: str):
        self.soup = soup
    
    def extract_descricao(self) -> str:
        descricao_elem = self.soup.find('p', {'data-testid': 'description-content'})
        descricao = descricao_elem.get_text(strip=True) if descricao_elem else "N/A"
        return descricao

In [28]:
descricao = ZapImoveisDescricao(soup_dados).extract_descricao()
print("Descrição:", descricao)

Descrição: Você sempre sonhou em ter o seu próprio cantinho para chamar de lar? Então esta é a oportunidade que você esperava!Esta linda casa de 3 quartos, sendo 1 suíte, é o lugar perfeito para a sua família. Com uma boa sala, 2 banheiros mais 1 lavabo, cozinha ampla e arejada, e 2 vagas de garagem, esta casa é ideal para quem busca conforto e comodidade.Com 115,45 m² de área, você terá espaço de sobra para viver momentos inesquecíveis com seus entes queridos. Além disso, o imóvel conta com um belo quintal, perfeito para relaxar e aproveitar.Localizada em um bairro tranquilo e seguro, próximo a escolas, supermercados e com fácil acesso a transporte público, esta casa é o lugar perfeito para você construir suas memórias.Aceita financiamento e FGTS.Não perca tempo, agende já a sua visita e se encante com esta maravilhosa casa. Venha fazer parte dessa história!Fale conosco: -


In [30]:
import logging
from typing import Optional

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p"
)

logger = logging.getLogger(__name__)


from typing import TypedDict, Dict

class ZapImoveisDados(TypedDict):
    titulo: str
    valor_venda: float
    metragem: str
    data_publicacao: str
    valor_condominio: str
    iptu: str
    cidade: str
    estado: str
    rua: str
    bairro: str
    quartos: str
    banheiros: str
    vagas: str
    descricao: str
    caracteristicas: Dict[str, str]
    url: str
    url_imagem: List[str]

class ZapImoveisPegandoDados:
    def __init__(self, link: str):
        self.link = link
        self.soup = self.pegando_soup()
        self.classe_dados_imovel = ZapImoveisDados
        self.link_imagem  = ZapImoveisPhotoLinks
        self.amenities = ZapImoveisAmenities
        self.metragem = ZapImoveisMetragem
        self.quartos = ZapImoveisQuartos
        self.banheiros = ZapImoveisBanheiros
        self.garagen = ZapImoveisVagasGaragem
        self.endereco = ZapImoveisEndereco
        self.titulo = ZapImoveisTitulo
        self.data_pubicacao = ZapImoveisDataPublicacao
        self.valor_venda = ZapImoveisValorVenda
        self.valor_condominio = ZapImoveisValorCondominio
        self.valor_iptu = ZapImoveisIPTU
        self.caracteristicas = ZapImoveisCaracteristicas
        self.descricao = ZapImoveisDescricao
        
    def pegando_soup(self) -> Optional[BeautifulSoup]:
        try:
            inst = ZapImoveisScraperBase(headless=True)
            html = inst.get_page_source(self.link)
            soup = BeautifulSoup(html, 'html.parser')
            logger.info("Soup obtido com sucesso: %s ", self.link)
            return soup
        except Exception as e:
            logger.error("Erro ao pegar os dados do link %s e o erro foi: %s", self.link,e)
            return None
    
    def pegar_amenities(self):
        soup = self.soup
        amenities = self.amenities(soup).extract_amenities()
        return amenities
    
    def pegando_link_photos(self) -> List:
        soup = self.soup
        try:
            fotos = self.link_imagem(soup).extract_photos()
            return fotos
        except Exception as e:
            logger.error("Erro ao pegar as fotos do anuncio %s e o erro foi: %s", {self.link},{e})
            return []
    
    def pegar_metragem(self) -> str:
        try:
            amenities = self.pegar_amenities()
            metragem = self.metragem(amenities).limpar_metragem()
            return metragem
        except Exception as e:
            logger.error("Erro ao pegar a metragem do anuncio %s e o erro foi: %s", {self.link},{e})
            return ""
        
    def pegar_quartos(self) -> str:
        
        try:
            amenities = self.pegar_amenities()
            quartos = self.quartos(amenities).extract_quartos() 
            return quartos
        except Exception as e:
            logger.error("Erro ao pegar os quartos do anuncio %s e o erro foi: %s", {self.link},{e})
            return ""
    
    def pegar_banheiros(self) -> str:
        try:
            amenities = self.pegar_amenities()
            banheiro = self.banheiros(amenities).extract_banheiros() 
            return banheiro
        except Exception as e:
            logger.error("Erro ao pegar os banheiros do anuncio %s e o erro foi: %s", {self.link},{e})
            return ""
        
    def pegar_vagas_garagen(self) -> str:
        try:
            amenities = self.pegar_amenities()
            vagas = self.garagen(amenities).extract_vagas()
            return vagas
        except Exception as e:
            logger.error("Erro ao pegar as vagas de garagem do anuncio %s e o erro foi: %s", {self.link},{e})
            return ""
        
    def pegar_enderecos(self):
        try:
            soup = self.soup
            rua, bairro, cidade, estado = self.endereco(soup).endereco_completo()
            return rua, bairro, cidade, estado
        except Exception as e:
            logger.error("Erro ao pegar o endereco do anuncio %s e o erro foi: %s", {self.link},{e})
            return "", "", "", ""
        
    def pegar_titulo(self):
        try:
            soup = self.soup
            titulo = self.titulo(soup).extract_titulo()
            return titulo
        except Exception as e:
            logger.error("Erro ao pegar o titulo do anuncio %s e o erro foi: %s", {self.link},{e})
            return ""
        
    def pegar_data_publicacao(self):
        try:
            soup = self.soup
            data_pulicacao = self.data_pubicacao(soup).converter_para_data()
            return data_pulicacao
        except Exception as e:
            logger.error("Erro ao pegar o data publicacao do anuncio %s e o erro foi: %s", {self.link},{e})
            return ""
        
    def pegar_valor_venda(self):
        try:
            soup = self.soup
            valor_venda = self.valor_venda(soup).limpar_valor_venda()
            return valor_venda
        except Exception as e:
            logger.error("Erro ao pegar o valor de venda do anuncio %s e o erro foi: %s", {self.link},{e})
            return None
        
    def pegar_valor_condominio(self):
        try:
            soup = self.soup
            valor_condominio = self.valor_condominio(soup).limpar_valor_condominio()
            return valor_condominio
        except Exception as e:
            logger.error("Erro ao pegar o valor do condominio anuncio %s e o erro foi: %s", {self.link},{e})
            return ""
        
    def pegar_valor_iptu(self):
        try:
            soup = self.soup
            valor_iptu = self.valor_iptu(soup).limpar_valor_iptu()
            return valor_iptu
        except Exception as e:
            logger.error("Erro ao pegar o valor do IPTU anuncio %s e o erro foi: %s", {self.link},{e})
            return ""
        
    def pegar_caracteristicas(self):
        try:
            soup = self.soup
            caracteristicas = self.caracteristicas(soup).extract_caracteristicas()
            return caracteristicas
        except Exception as e:
            logger.error("Erro ao pegar as caracteristica do anuncio %s e o erro foi: %s", {self.link},{e})
            return {}
        
    def pegar_descricao(self):
        try:
            soup = self.soup
            descricao = self.descricao(soup).extract_descricao()
            return descricao
        except Exception as e:
            logger.error("Erro ao pegar as descricoes do anuncio %s e o erro foi: %s", {self.link},{e})
            return ""
       
    def extrair_todos_dados(self) -> Optional[ZapImoveisDados]:
        """Extrai todos os dados do imóvel de uma vez"""
        logger.info("Iniciando extração completa %s", self.link)
        
        if self.soup is None:
            logger.error("Não foi possível obter o soup para %s," , self.link)
            return None
        
        try:
            # Extrair endereço
            rua, bairro, cidade, estado = self.pegar_enderecos()
            
            # Montar dicionário completo
            dados = ZapImoveisDados(
                titulo=self.pegar_titulo(),
                valor_venda=self.pegar_valor_venda(),
                metragem=self.pegar_metragem(),
                data_publicacao=self.pegar_data_publicacao(),
                valor_condominio=self.pegar_valor_condominio(),
                iptu=self.pegar_valor_iptu(),
                cidade=cidade,
                estado=estado,
                rua=rua,
                bairro=bairro,
                quartos=self.pegar_quartos(),
                banheiros=self.pegar_banheiros(),
                vagas=self.pegar_vagas_garagen(),
                descricao=self.pegar_descricao(),
                caracteristicas=self.pegar_caracteristicas(),
                url=self.link,
                url_imagem=self.pegando_link_photos(),
            )
            
            logger.info("Extração concluída com sucesso %s",self.link)
            return dados
            
        except Exception as e:
            logger.error("Erro ao extrair todos os dados do link %s:%s ",self.link, e, exc_info=True)
            return None
     

In [ ]:
zip_imoveis_dados = []
for link in links:
    zap_dados = ZapImoveisPegandoDados(link)
    dados_zep_imoveis= zap_dados.extrair_todos_dados()
    zip_imoveis_dados.append(dados_zep_imoveis)
zip_imoveis_dados

In [224]:
import logging
from typing import List, Dict, Optional, Any, Callable
from dataclasses import dataclass, asdict
from functools import cached_property

# Configuração de Log centralizada
logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s [%(levelname)s] %(message)s", 
    datefmt="%m/%d/%Y %I:%M:%S %p"
)
logger = logging.getLogger(__name__)

@dataclass
class ZapImoveisDados:
    titulo: str
    valor_venda: Optional[float]
    metragem: str
    data_publicacao: str
    valor_condominio: str
    iptu: str
    cidade: str
    estado: str
    rua: str
    bairro: str
    quartos: str
    banheiros: str
    vagas: str
    descricao: str
    caracteristicas: Dict[str, str]
    url: str
    url_imagem: List[str]

class ZapImoveisPegandoDados:
    def __init__(self, link: str):
        self.link = link
        # O soup é carregado na primeira vez que for usado
        
    @cached_property
    def soup(self) -> Optional[Any]:
        """Obtém o soup apenas uma vez e faz cache do resultado."""
        try:
            inst = ZapImoveisScraperBase(headless=True)
            html = inst.get_page_source(self.link)
            if not html:
                return None
            logger.info("Soup obtido com sucesso: %s", self.link)
            return BeautifulSoup(html, 'html.parser')
        except Exception as e:
            logger.error("Erro ao obter soup de %s: %s", self.link, e)
            return None

    @cached_property
    def amenities_data(self):
        """Cache para evitar reprocessar amenities múltiplas vezes."""
        if not self.soup: return None
        return ZapImoveisAmenities(self.soup).extract_amenities()

    def _safe_extract(self, func: Callable, default: Any = "", *args) -> Any:
        """Helper para evitar repetição de try/except em todos os métodos."""
        try:
            return func(*args)
        except Exception as e:
            logger.error("Erro em %s para o link %s: %s", func.__name__, self.link, e)
            return default

    # --- Métodos de Extração Refatorados ---

    def pegar_metragem(self) -> str:
        return self._safe_extract(lambda: ZapImoveisMetragem(self.amenities_data).limpar_metragem())

    def pegar_quartos(self) -> str:
        return self._safe_extract(lambda: ZapImoveisQuartos(self.amenities_data).extract_quartos())

    def pegar_banheiros(self) -> str:
        return self._safe_extract(lambda: ZapImoveisBanheiros(self.amenities_data).extract_banheiros())

    def pegar_vagas_garagem(self) -> str:
        return self._safe_extract(lambda: ZapImoveisVagasGaragem(self.amenities_data).extract_vagas())

    def pegar_enderecos(self):
        # Retorno padrão para desempacotamento seguro
        return self._safe_extract(
            lambda: ZapImoveisEndereco(self.soup).endereco_completo(),
            default=("", "", "", "")
        )

    def pegar_valor_venda(self) -> Optional[float]:
        return self._safe_extract(lambda: ZapImoveisValorVenda(self.soup).limpar_valor_venda(), default=None)

    def pegar_fotos(self) -> List[str]:
        return self._safe_extract(lambda: ZapImoveisPhotoLinks(self.soup).extract_photos(), default=[])

    def extrair_todos_dados(self) -> Optional[ZapImoveisDados]:
        if not self.soup:
            return None

        logger.info("Iniciando extração completa: %s", self.link)
        
        rua, bairro, cidade, estado = self.pegar_enderecos()

        try:
            dados = ZapImoveisDados(
                titulo=self._safe_extract(lambda: ZapImoveisTitulo(self.soup).extract_titulo()),
                valor_venda=self.pegar_valor_venda(),
                metragem=self.pegar_metragem(),
                data_publicacao=self._safe_extract(lambda: ZapImoveisDataPublicacao(self.soup).converter_para_data()),
                valor_condominio=self._safe_extract(lambda: ZapImoveisValorCondominio(self.soup).limpar_valor_condominio()),
                iptu=self._safe_extract(lambda: ZapImoveisIPTU(self.soup).limpar_valor_iptu()),
                cidade=cidade,
                estado=estado,
                rua=rua,
                bairro=bairro,
                quartos=self.pegar_quartos(),
                banheiros=self.pegar_banheiros(),
                vagas=self.pegar_vagas_garagem(),
                descricao=self._safe_extract(lambda: ZapImoveisDescricao(self.soup).extract_descricao()),
                caracteristicas=self._safe_extract(lambda: ZapImoveisCaracteristicas(self.soup).extract_caracteristicas(), {}),
                url=self.link,
                url_imagem=self.pegar_fotos()
            )
            logger.info("Extração concluída com sucesso: %s", self.link)
            return dados
        except Exception as e:
            logger.error("Falha crítica na extração de %s: %s", self.link, e, exc_info=True)
            return None

In [ ]:
zip_imoveis_dados = []
for link in links:
    zap_dados = ZapImoveisPegandoDados(link)
    dados_zep_imoveis= zap_dados.extrair_todos_dados()
    zip_imoveis_dados.append(dados_zep_imoveis)

In [ ]:
pd.DataFrame(zip_imoveis_dados)

In [ ]:
import asyncio
import logging
from typing import List, Dict, Optional, Any, Callable
from dataclasses import dataclass
from functools import partial

# Configuração de Log
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

@dataclass
class ZapImoveisDados:
    titulo: str
    valor_venda: Optional[float]
    metragem: str
    data_publicacao: str
    valor_condominio: str
    iptu: str
    cidade: str
    estado: str
    rua: str
    bairro: str
    quartos: str
    banheiros: str
    vagas: str
    descricao: str
    caracteristicas: Dict[str, str]
    url: str
    url_imagem: List[str]


    


In [232]:
class ZapImoveisPegandoDadosAsync:
    def __init__(self, link: str):
        self.link = link
        self.soup = None
        self._amenities_cache = None

    async def inicializar_soup(self):
        """Método assíncrono para carregar o HTML."""
        try:
            # Se o ScraperBase for síncrono, rodamos em um thread pool para não travar o loop
            loop = asyncio.get_event_loop()
            inst = ZapImoveisScraperBase(headless=True)
            
            # Executa a chamada bloqueante do Selenium sem travar o programa
            html = await loop.run_in_executor(None, inst.get_page_source, self.link)
            
            if html:
                self.soup = BeautifulSoup(html, 'html.parser')
                # Pré-processa amenities para cache
                self._amenities_cache = ZapImoveisAmenities(self.soup).extract_amenities()
                logger.info(f"Dados carregados: {self.link}")
        except Exception as e:
            logger.error(f"Erro ao carregar {self.link}: {e}")

    def _safe_extract(self, func: Callable, default: Any = "", *args) -> Any:
        """Wrapper síncrono para extração de dados do Soup já carregado."""
        try:
            return func(*args)
        except Exception as e:
            return default

    async def extrair_todos_dados(self) -> Optional[ZapImoveisDados]:
        """Orquestra a extração após o soup estar pronto."""
        if not self.soup:
            await self.inicializar_soup()
        
        if not self.soup:
            return None

        # Como o processamento do Soup é CPU-bound e rápido, 
        # não precisamos de 'await' em cada pequeno campo.
        try:
            rua, bairro, cidade, estado = self._safe_extract(
                lambda: ZapImoveisEndereco(self.soup).endereco_completo(),
                ("", "", "", "")
            )

            return ZapImoveisDados(
                titulo=self._safe_extract(lambda: ZapImoveisTitulo(self.soup).extract_titulo()),
                valor_venda=self._safe_extract(lambda: ZapImoveisValorVenda(self.soup).limpar_valor_venda(), None),
                metragem=self._safe_extract(lambda: ZapImoveisMetragem(self._amenities_cache).limpar_metragem()),
                # ... (outros campos seguindo o mesmo padrão)
                url=self.link,
                url_imagem=self._safe_extract(lambda: ZapImoveisPhotoLinks(self.soup).extract_photos(), [])
            )
        except Exception as e:
            logger.error(f"Erro na extração final de {self.link}: {e}")
            return None

# --- Como executar múltiplos links em paralelo ---

async def processar_lista_de_links(links: List[str]):
    tasks = []
    for link in links:
        scraper = ZapImoveisPegandoDadosAsync(link)
        tasks.append(scraper.extrair_todos_dados())
    
    # Executa todos os scrapers simultaneamente
    resultados = await asyncio.gather(*tasks)
    return [r for r in resultados if r is not None]

In [234]:
dados = await processar_lista_de_links(links)

02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager ======
02/10/2026 09:23:48  ====== WebDriver manager 